In [13]:
import pandas as pd

survey = pd.read_excel('ABE Career Outcomes Data 2016-2021.xlsx')
survey

,Degree Date,Degree Level,Major 1 at Graduation,Organization Name,Job Title,City,State
0,2016-12-01,Bachelor's,Agricultural Engineering,Double L Group,Design Engineer,Dyersville,Iowa
1,2016-12-01,Bachelor's,Agricultural Engineering,John Deere,Design Engineering Developmental Program,Waterloo,Iowa
2,2016-12-01,Bachelor's,Agricultural Engineering,John Deere,Engineer,Ankeny,Iowa
3,2016-12-01,Bachelor's,Agricultural Engineering,John Deere,Marketing Rep,Moline,Illinois
4,2016-12-01,Bachelor's,Agricultural Engineering,John Deere,Quality Engineering Development Program,Waterloo,Iowa
...,...,...,...,...,...,...,...
373,2021-05-01,Bachelor's,Biological Systems Engineering,NaN,NaN,NaN,NaN
374,2021-05-01,Bachelor's,Biological Systems Engineering,NaN,NaN,NaN,NaN
375,2021-05-01,Bachelor's,Biological Systems Engineering,NaN,NaN,NaN,NaN
376,2021-05-01,Bachelor's,Biological Systems Engineering,NaN,NaN,NaN,NaN


In [14]:
survey['Organization Name'].value_counts()

John Deere                      18
Vermeer Corporation             12
Cargill, Incorporated            8
Kuhn North America, Inc.         5
Henning Companies LLC            5
                                ..
Toyne Inc.                       1
Puck Custom Enterprises          1
Renewable Energy Group (REG)     1
WSP                              1
Universal Industries, Inc.       1
Name: Organization Name, Length: 100, dtype: int64

In [57]:
bse = survey[survey['Major 1 at Graduation'] == 'Biological Systems Engineering']
ag = survey[survey['Major 1 at Graduation'] == 'Agricultural Engineering']
print('TOP 10 EMPLOYERS FOR BSE AND AE \n \nBSE Common Employers:\n',bse['Organization Name'].value_counts().nlargest(10))
print('AE Common Employers: \n', ag['Organization Name'].value_counts().nlargest(10))
abe = pd.concat([bse['Organization Name'],ag['Organization Name']], axis=0)
abe = survey['Organization Name']
abe = abe.dropna(axis=0)
abe = abe.unique()
abe = pd.DataFrame(abe, columns=['Employer'])
display(abe)

TOP 10 EMPLOYERS FOR BSE AND AE 
 
BSE Common Employers:
 Cargill, Incorporated                                                                                        6
ISG                                                                                                          2
John Deere                                                                                                   2
Ardent Mills                                                                                                 2
PepsiCo                                                                                                      2
ADM                                                                                                          2
Koch Industries Companies (Georgia-Pacific, Molex, Guardian Industries, Flint Hills Resources, & INVISTA)    1
Ajinomoto North America, Inc                                                                                 1
Tate & Lyle                                           

,Employer
0,Double L Group
1,John Deere
2,Kent Corporation
3,Syngenta
4,"Cargill, Incorporated"
...,...
95,United States Department of Agriculture
96,3M
97,"Frito-Lay, Inc."
98,"JEO Consulting Group, Inc."


In [59]:
# Next, I isolated the most common job titles that ABE graduates have. 
# I selected all job titles that at least 2 people acquired upon graduation. 
bse_job = bse['Job Title'].value_counts().nlargest(4).to_frame()
ag_job = ag['Job Title'].value_counts().nlargest(17).to_frame()
bse_joblist = bse_job.index
ag_joblist = ag_job.index
abe_joblist = pd.concat(bse_joblist, ag_joblist, axis=1)

TypeError: concat() got multiple values for argument 'axis'

In [17]:
## ***************************
# huge thank you to Ryan Jeon for developing the base of this indeed text scraping code
# see supplemental information for the source code created by him, it has been modified slightly for this application
from bs4 import BeautifulSoup
import requests

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
df = pd.DataFrame(columns = ["Job_Titles"])
df2 = pd.DataFrame(columns = ["Company"])
df3 = pd.DataFrame(columns = ["URL"])


r = requests.get('https://www.indeed.com/jobs?q=agricultural%20engineer&start=10'.format(0), headers = headers)
soup = BeautifulSoup(r.text, 'html.parser')
titles = soup.select("h2 span") 
# select all span tags under the umbrella of h2 tags 
companies = soup.find_all(class_ = "companyName")
URLs = soup.find_all('a', attrs = {'class' : 'tapItem'})

for title in titles:
    titles_list = title.text
    # the gotcha here is that there are "news" scattered so we have to remove those first before concatenating our data
    df.loc[len(df.index)] = [titles_list]
    df = df[df.Job_Titles != "new"]

for company in companies:
    company_list = company.text
    df2.loc[len(df2.index)] = [company_list]

for URL in URLs:
    base = 'http://www.indeed.com'
    link = URL.attrs['href']
    new_URL = base + link
    df3.loc[len(df3.index)] = [new_URL]    
    
indeed = pd.concat([df, df2, df3], axis=1, join='inner')
display(df)
display(indeed)
## *************** end of code edited from Ryan Jeon's amazing work! :)

,Job_Titles


,Job_Titles,Company,URL


In [18]:
df3['URL']

Series([], Name: URL, dtype: object)

In [19]:
URL = indeed['URL'][1]
page = requests.get(URL.format(0), headers = headers) 
soup = BeautifulSoup(page.text, 'html.parser')
posting = soup.get_text()

IndexError: index 1 is out of bounds for axis 0 with size 0

In [ ]:
jdesc =  pd.DataFrame(columns = ["Job Description"])

for post in indeed['URL']:
    r = requests.get(post.format(0), headers = headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    job_description = soup.find('div',{'id':'jobDescriptionText'})
    jd = job_description.text if job_description else "N/A"
    jdesc.loc[len(jdesc.index)] = [jd.replace('\n', ' ')]

In [ ]:
jdesc

In [77]:
def indeed_posts(search_term):

    kw = search_term.split(" ")
 #   search_url = 'https://www.indeed.com/jobs?q='
 #   for word in range(0,len(kw)-1):
 #       search_url = search_url + kw[word] + '%20'
 #   search_url = search_url + kw[len(kw)-1] + '&start={' + pagenumber + '}'
    ## *************************** a huge thank you to Ryan Jeon for developing the base of this indeed text scraping code
    # see supplemental information for the source code created by him, it has been modified slightly for this application
    from bs4 import BeautifulSoup
    import requests
    import time
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15'}
    df = pd.DataFrame(columns = ["Job_Titles"])
    df2 = pd.DataFrame(columns = ["Company"])
    df3 = pd.DataFrame(columns = ["URL"])
    sind = f'https://www.indeed.com/jobs?as_and='
    kword = '&as_phr&as_any=biological%2C%20agriculture%2C%20food%2C%20environment%2C%20biofuel%2C%20fermentation%2C%20water%2C%20machinery%2C%20animal'
    end = '&as_not&as_ttl&as_cmp&jt=all&st&salary&radius=25&l&fromage=any&limit=10&sort&psf=advsrch&from=advancedsearch&vjk=ffced369343eb245'
    # key words to look for in job title/description to get closer to the industry students join
    for pagenum in range(0,30,10):
        for word in range(0,len(kw)-1):
            search_url = sind + kw[word] + '%20'
        search_url = search_url + kw[len(kw)-1] + kword + end+'%20&start=' + str(pagenum)
        r = requests.get(search_url, headers = headers)
        time.sleep(1)
        soup = BeautifulSoup(r.text, 'html.parser')
        titles = soup.select("h2 span") 
        # select all span tags under the umbrella of h2 tags 
        companies = soup.find_all(class_ = "companyName")
        URLs = soup.find_all('a', attrs = {'class' : 'tapItem'})
        for title in titles:
            titles_list = title.text
            # the gotcha here is that there are "news" scattered so we have to remove those first before concatenating our data
            df.loc[len(df.index)] = [titles_list]
            df = df[df.Job_Titles != "new"]
        for company in companies:
            company_list = company.text
            df2.loc[len(df2.index)] = [company_list]
        for URL in URLs:
            base = 'http://www.indeed.com'
            link = URL.attrs['href']
            new_URL = base + link
            df3.loc[len(df3.index)] = [new_URL]    
        ## *************************** end of code edited from Ryan Jeon's amazing work! :)

    jdesc =  pd.DataFrame(columns = ["Job Description"])

    for post in df3['URL']:
        r = requests.get(post.format(0), headers = headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        job_description = soup.find('div',{'id':'jobDescriptionText'})
        jd = job_description.text if job_description else "N/A"
        jdesc.loc[len(jdesc.index)] = [jd.replace('\n', ' ')]   
    indeed = pd.concat([df, df2, df3, jdesc], axis=1, join='inner')
    return indeed

In [76]:
y = indeed_posts('Project Engineer')
display(y)

https://www.indeed.com/jobs?as_and=Project%20Engineer&as_phr&as_any=biological%2C%20agriculture%2C%20food%2C%20environment%2C%20biofuel%2C%20fermentation%2C%20water%2C%20machinery%2C%20animal&as_not&as_ttl&as_cmp&jt=all&st&salary&radius=25&l&fromage=any&limit=10&sort&psf=advsrch&from=advancedsearch&vjk=ffced369343eb245%20&start=0
https://www.indeed.com/jobs?as_and=Project%20Engineer&as_phr&as_any=biological%2C%20agriculture%2C%20food%2C%20environment%2C%20biofuel%2C%20fermentation%2C%20water%2C%20machinery%2C%20animal&as_not&as_ttl&as_cmp&jt=all&st&salary&radius=25&l&fromage=any&limit=10&sort&psf=advsrch&from=advancedsearch&vjk=ffced369343eb245%20&start=10
https://www.indeed.com/jobs?as_and=Project%20Engineer&as_phr&as_any=biological%2C%20agriculture%2C%20food%2C%20environment%2C%20biofuel%2C%20fermentation%2C%20water%2C%20machinery%2C%20animal&as_not&as_ttl&as_cmp&jt=all&st&salary&radius=25&l&fromage=any&limit=10&sort&psf=advsrch&from=advancedsearch&vjk=ffced369343eb245%20&start=20
ht

,Job_Titles
0,Project Engineer II
1,MECHICAL ENGINEER (NO EXPERIENCE)
2,Mechanical Designer / Engineer-Work Remotely!
3,MECHANICAL ENGINEER (NO EXPERIENCE)
4,Project Engineer
...,...
85,Research & Development Design Engineer
86,Water Resource Engineer
87,Engineer Assistant
88,Project Engineer - Food & Beverage Process


,Job_Titles,Company,URL,Job Description
0,Project Engineer II,Cargill,http://www.indeed.com/rc/clk?jk=ffced369343eb2...,"Want to build a stronger, more sustainable fut..."
1,MECHICAL ENGINEER (NO EXPERIENCE),CEMEX USA,http://www.indeed.com/company/CEMEX-USA/jobs/M...,"Company OverviewFounded in 1906, over the last..."
2,Mechanical Designer / Engineer-Work Remotely!,SSI Consulting,http://www.indeed.com/company/SSI-Consulting/j...,We are seeking mechanical designers and / or e...
3,MECHANICAL ENGINEER (NO EXPERIENCE),CEMEX USA,http://www.indeed.com/company/CEMEX-USA/jobs/M...,"Company OverviewFounded in 1906, over the last..."
4,Project Engineer,Jackson Dean Construction,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,Project EngineerJackson Dean Construction has ...
...,...,...,...,...
85,Research & Development Design Engineer,Johnson & Johnson Family of Companies,http://www.indeed.com/rc/clk?jk=4fe8b85c9e4b55...,"Ethicon Endo Surgery Inc., a member of Johnson..."
86,Water Resource Engineer,Colorado Springs Utilities,http://www.indeed.com/rc/clk?jk=27732ed09af3f0...,Job Type: Engineering/Engineering Support Pay ...
87,Engineer Assistant,"City of La Vista, NE",http://www.indeed.com/rc/clk?jk=ae4b20b3600ade...,Applications for a full-time Engineer Assistan...
88,Project Engineer - Food & Beverage Process,Stellar Group,http://www.indeed.com/rc/clk?jk=874b91788294a4...,Overview: Stellar is seeking a Food & Beverag...


In [78]:
y['Job Description']

0     Want to build a stronger, more sustainable fut...
1     Company OverviewFounded in 1906, over the last...
2     We are seeking mechanical designers and / or e...
3     Company OverviewFounded in 1906, over the last...
4     Project EngineerJackson Dean Construction has ...
                            ...                        
85    Ethicon Endo Surgery Inc., a member of Johnson...
86    Job Type: Engineering/Engineering Support Pay ...
87    Applications for a full-time Engineer Assistan...
88    Overview:  Stellar is seeking a Food & Beverag...
89    RFA Engineering (www.rfamec.com) is seeking a ...
Name: Job Description, Length: 90, dtype: object

In [79]:
from collections import Counter
Counter(" ".join(y['Job Description']).split()).most_common(100)

[('and', 3021),
 ('to', 1648),
 ('the', 1437),
 ('of', 1301),
 ('in', 894),
 ('a', 862),
 ('for', 623),
 ('with', 621),
 ('or', 531),
 ('is', 443),
 ('be', 391),
 ('will', 342),
 ('on', 325),
 ('are', 305),
 ('as', 303),
 ('an', 284),
 ('work', 270),
 ('our', 269),
 ('you', 247),
 ('engineering', 240),
 ('that', 192),
 ('your', 187),
 ('by', 185),
 ('experience', 175),
 ('this', 172),
 ('at', 171),
 ('project', 168),
 ('from', 154),
 ('We', 150),
 ('all', 149),
 ('design', 148),
 ('other', 137),
 ('we', 132),
 ('Engineer', 132),
 ('have', 130),
 ('including', 127),
 ('The', 124),
 ('projects', 119),
 ('may', 118),
 ('not', 117),
 ('technical', 116),
 ('&', 114),
 ('Engineering', 108),
 ('team', 101),
 ('required', 96),
 ('position', 95),
 ('This', 90),
 ('support', 90),
 ('development', 90),
 ('new', 87),
 ('and/or', 84),
 ('related', 83),
 ('time', 77),
 ('working', 76),
 ('product', 76),
 ('You', 76),
 ('equipment', 75),
 ('years', 73),
 ('must', 70),
 ('Experience', 69),
 ('manufact

In [ ]:
search_string = "quality assurance engineer"
kw = search_string.split(" ")
print(kw)
print(len(kw))
search_url = 'https://www.indeed.com/jobs?q='
for word in range(0,len(kw)-1):
    search_url = search_url + kw[word] + '%20'
search_url = search_url + kw[len(kw)-1] + '&start={}'
search_url
